In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv("../input/uniqlo-fastretailing-stock-price-prediction/Uniqlo(FastRetailing) 2012-2016 Training - stocks2012-2016.csv")

In [ ]:
dataset.head()

In [ ]:
dataset.isna().any()

In [ ]:
dataset.info()

In [ ]:
dataset['Open'].plot(figsize=(16,6))

In [ ]:
#  rolling mean
dataset.rolling(7).mean().head(20)

In [ ]:
dataset['Open'].plot(figsize=(16,6))
dataset.rolling(window=10).mean()['Close'].plot()

In [ ]:
#optional specify a minimum number of periods
dataset['Close'].expanding(min_periods=1).mean().plot(figsize=(16,6))

In [ ]:
training_set = dataset['Open']
training_set = pd.DataFrame(training_set)

In [ ]:
dataset.isna().any()

In [ ]:
#feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
X_train=[]
y_train=[]
for i in range(60, 1225):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

#reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


In [ ]:

#initialising the RNN
regressor = Sequential()

In [ ]:
#adding the first  LSTM and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

#adding a second LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))

#adding a third LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

#adding a fourth LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

#adding the output layer
regressor.add(Dense(units = 1))




In [ ]:
#making the predictions and visualising the results
dataset_test = pd.read_csv("../input/uniqlo-fastretailing-stock-price-prediction/Uniqlo(FastRetailing) 2012-2016 Training - stocks2012-2016.csv")

In [ ]:
#getting the real stock price of 2016
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
dataset_test.head()

In [ ]:
dataset_test.info()

In [ ]:
test_set = dataset_test['Open']
test_set = pd.DataFrame(test_set)

In [ ]:
test_set.info()

In [ ]:
#getting the predicted stock price of 2016
dataset_total= pd.concat((dataset['Open'],dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60,80):
    
    X_test.append(inputs[i-60:i,0])
X_test= np.array(X_test)
X_test= np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price= pd.DataFrame(predicted_stock_price)
predicted_stock_price.info()

In [ ]:
#visualizing the results
plt.plot(real_stock_price, color ='red', label = 'Real Stock Price')
plt.plot(predicted_stock_price,color = 'blue' , label= 'Predicted Stock Price')
plt.title('Uniqlo FastRetailing Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Uniqlo Stock Price')
plt.legend()
plt.show()